## 3: Language Style

Playing with language and style

As before, we'll start with the imports

In [2]:
import os, openai, json

Now we need to set the API key and (if applicable) the organization you belong to if that's set up. You can manually put them in or bring them in from the environement.

In [3]:
openai.api_key = os.environ['OPENAI_KEY']

In [4]:
try:
    openai.organization = os.environ['OPENAI_ORG']
    print('Organization Added')
except:
    print('No Organization')

Organization Added


In [5]:
#Run this if you want to use your personal instead of default org
openai.organization=None

To call openai, we need to do an openai.Completion.create() with a bunch of parameters. To make this easier, we'll create a wrapper function that (1) has some default parameters as well as (2) saves each request/response to file for us.

Let's start by creating a folder to drop prompts

In [6]:
os.makedirs('prompts', exist_ok=True)

now we'll actually make the wrapper function that can take args as well as has defaults

In [7]:
def query(prompt, full=False, **kwargs):
    modelKwargs = {
        'temperature':0,
        'engine':'davinci',
        'max_tokens':10,
        'stop':['\n']
    }
    
    for kwarg in kwargs:
        modelKwargs[kwarg] = kwargs[kwarg]
    
    completion = openai.Completion.create(prompt=prompt, **modelKwargs)
    
    with open('prompts/{}.json'.format(completion['id']), 'w') as fh:
        json.dump(completion, fh, indent=4)
    
    if full:
        return completion
    if 'n' in modelKwargs:
        return [x['text'] for x in completion['choices']]
    return completion['choices'][0]['text']

let's make sure it works!

In [8]:
query('1+1=')

'2.'

One of the cool things that GPT learned how to do is language translation. We can do simple translation simply by telling it to translate into French or German. (These may not be great, I don't know lol)

In [9]:
prompt = """-Translations-
English: Where is the library?
French:"""

In [10]:
print(query(prompt))

 Où est la bibliothèque


In [11]:
prompt = """-Translations-
English: Where is the library?
German:"""

In [12]:
print(query(prompt))

 Wo ist das Bibliothek?


How's it do on idioms?

In [15]:
prompt = """-Translations-
English: Don't worry, water under the bridge
French:"""

In [18]:
print(query(prompt, max_tokens=30))

 Ne t'en fais pas, ça va passer


Cool! (for what it's worth, try that on Google translate). Of course, depending on how you word it, it might do it literally.

So what about style? Well, sometimes we want to reword stuff. We did this before with Math if you remember; we came up with one representation of numbers and then translated that into another representation as so:

In [19]:
prompt = """Task: Spell out the sum of each math problem.

Question: 40 + 50
Answer: 90
Spelled Out: Ninety

Question: 1+1
Answer: 2
Spelled Out: Two

Question: 50 + 80
Answer: 130
Spelled Out: One hundred and twenty

Question: 12 + 52
Answer: 64
Spelled Out: Sixty Four

Question: {}
Answer:"""

print(query(prompt.format('51+12'), stop=['\n\n']))

 63
Spelled Out: Sixty Three



We can do this with language in general. So let's take a formal statement and make it slangy.

In [20]:
prompt = """Formal: Hello it is nice to you. My name is Mr. Bot.
Informal:"""

In [21]:
print(query(prompt))

 Hi, my name is Mr. Bot.


It can also change words based on context

In [44]:
prompt = """American English: I love how that elevator is my favorite color, yellow.
Re-written as said by a Brit:"""

In [45]:
print(query(prompt, max_tokens=30))

 I love how that lift is my favourite colour, yellow.


Oddly, that doesn't work for translating elevator to lift if we don't include the color and I don't know why, but we can fix that by giving a prefix of the task along with a completed example (1 shot).

In [47]:
prompt = """American English: I love how that elevator is my favorite color.
Re-written as said by a Brit:"""
print(query(prompt, max_tokens=30))

 I love how that elevator is my favourite colour.


In [51]:
prompt = """People in different places spell/say things differently.

American: Look, a truck!
Brit: Look, a lorry!

American: I love how that elevator is my favorite color.
Brit:"""
print(query(prompt, max_tokens=30))

 I love how that lift is my favourite colour.


We can also use it to re-word concepts

In [53]:
prompt = """Sometimes you need to re-word things so kids can understand them.

Original: Man the tax rate is killing me right now.
Reworded for Kids: We have to give the government a lot of money

Original: A lot of people shorted GME.
Reworded for Kids:"""
print(query(prompt, max_tokens=30))

 A lot of people bet that GME would go down.


In [57]:
prompt = """Sometimes you need to re-word things so kids can understand them.

Original: Man the tax rate is killing me right now.
Reworded for Kids: We have to give the government a lot of money

Original: A lot of people shorted GME. However, other people squeezed the stock to make them cover.
Reworded for Kids:"""
print(query(prompt, max_tokens=30))

 A lot of people bet against GME. However, other people made them pay up.


In [62]:
prompt = """Sometimes you need to re-word things so kids can understand them.

Original: Man the tax rate is killing me right now.
Reworded for Kids: We have to give the government a lot of money

Original: My team lost but at at the last minute they scored to cover the spread.
Reworded for Kids:"""
print(query(prompt, max_tokens=30))

 My team lost but at the last minute they scored to make the score closer to what the bookies predicted.


Depending on what you want your outputs to look like you might need multiple tries. We can do cool stuff with this sort of thing in different contexts. E.g. let's take some tweets from CNN and Fox and see what happens if we use them as examples.

inputs:
Biden Nato Speech:

https://www.cnn.com/2021/06/14/politics/joe-biden-phony-populism-nato/index.html

https://www.foxnews.com/politics/biden-trump-republicans-nato-foreign-soil

In [64]:
prompt = """Event: Biden gives speech at NATO about fighting populism
CNN Headline: Biden says NATO must protect against 'phony populism'

Event: Biden gives speech at NATO about fighting populism
Fox Headline: Biden slams Republicans while on foreign soil in break with political norms

Event: {}
{} Headline"""

In [68]:
event = 'Congress passes tax breaks'
print(query(prompt.format(event, 'CNN')))
print(query(prompt.format(event, 'Fox')))

: Congress passes tax breaks for middle class
: Congress passes tax breaks for the rich


In [69]:
event = 'Immigration goes up'
print(query(prompt.format(event, 'CNN')))
print(query(prompt.format(event, 'Fox')))

: Immigration arrests up 38% under Trump
: Illegal immigration up 40% in 2017


In [72]:
event = 'Drought impacts California'
print(query(prompt.format(event, 'CNN')))
print(query(prompt.format(event, 'Fox')))

: California drought: Gov. Jerry Brown declares state
: California drought is a hoax, says California farmer


It's worth noting GPT3's training ended a couple years ago so some things got picked up on. Of course, there's all sorts of biases etc. that end up in these sets that are worth considering when using the model for different tasks. We can also get funner outputs using the temperature parameter, so we're not using the most likely next token each time.

In [79]:
event = 'France declares war on Spain'
print(query(prompt.format(event, 'CNN'), max_tokens=30, temperature=.5))
print(query(prompt.format(event, 'Fox'), max_tokens=30, temperature=.5))
print(query(prompt.format(event, 'BBC'), max_tokens=30, temperature=.5))
print(query(prompt.format(event, 'The Onion'), max_tokens=30, temperature=.5))

: U.S. must support France in war on Spain, Biden says
: France declares war on Spain after Spain refuses to allow refugees to enter
: France declares war on Spain
: French Military Prepared to Destroy Entire Nation of Spain


In [80]:
event = 'France declares war on Spain'
print(query(prompt.format(event, 'CNN'), max_tokens=30, temperature=.5))
print(query(prompt.format(event, 'Fox'), max_tokens=30, temperature=.5))
print(query(prompt.format(event, 'BBC'), max_tokens=30, temperature=.5))
print(query(prompt.format(event, 'The Onion'), max_tokens=30, temperature=.5))

: Report: France declares war on Spain
: France declares war on Spain
: France declares war on Spain
: France Declares War on Spain


We can get it to get really creative by increasing the temperature further

In [81]:
event = 'France declares war on Spain'
print(query(prompt.format(event, 'CNN'), max_tokens=30, temperature=1))
print(query(prompt.format(event, 'Fox'), max_tokens=30, temperature=1))
print(query(prompt.format(event, 'BBC'), max_tokens=30, temperature=1))
print(query(prompt.format(event, 'The Onion'), max_tokens=30, temperature=1))

: France declares war on Spain, says the government rightfully split from citizens meeting key demands
: France attacks the Pyrenees.
: Armistice is reached with Spain
: In Terrifying Sign Of Things To Come, ISIS Continues Unabated


At temperature about 1 it just goes way off track

In [82]:
event = 'France declares war on Spain'
print(query(prompt.format(event, 'CNN'), max_tokens=30, temperature=1.5))
print(query(prompt.format(event, 'Fox'), max_tokens=30, temperature=1.5))
print(query(prompt.format(event, 'BBC'), max_tokens=30, temperature=1.5))
print(query(prompt.format(event, 'The Onion'), max_tokens=30, temperature=1.5))

：France Declares war 5 different out of 75
: Right in city named japan places between the indians ill fated organization
:  Hungary expected to ship last thousands migrants after chaotic borders quota launch ======================================= SUPER POWER USA Back in History Jan -- Why must Elizabeth buck? 27
: It's Definitely Effin Illegal


So this gets to a more common theme: writing text in the style of someone else. Try it!

In [83]:
prompt = """
YOUR PROMPT HERE
"""